In [10]:
import pandas as pd
import re
import networkx as nx
import pylab

def sentiment_review(review, brand_list):
    review = re.sub("\"", "", review)
    # review = re.sub("")
    
    for item in brand_list:
        iter = re.finditer(item, review)
        if iter:
            sentiment = []
            for m in iter:
                sentence = review[max(0, m.start(0-70)): min((m.start(0) + len(item)+70), len(review))]
                                  
    return sent_score_fin

def sentiment_scores(reviews_data, brand_list):
    """
    gets the value of sentiment for each review for all the brands,
    PARAM reviews_data: Raw input of reviews data
    PARAM brand_list: list of brands to be considered
    RETURN: a dictionary of sentiment scores by key post
    """
    sent_scores = {}
    for item in brand_list:
        sent_scores[item]=list()
    for review_id in range(0, len(reviews_data)):
        sent_values = sentiment_review(reviews_data.iloc[review_id, 0], brand_list)
        for item in brand_list:
            sent_scores[item].append(sent_values[item])
    fin = pd.DataFrame.from_dict(sent_scores, orient='columns')
    fin.to_csv('Sentiment_Scores.csv', index=False)
    return sent_scores

def create_edges(sent_scores):
    """
    Calculates the edgeweight by row. If it is positive it is added
    to a list of positive sentiment, else to a list of negative sentiment.
    The edge from x->y is weighed as mean of positive sentiment list.
    The edge from y->x is weighed as absolut evalue of the mean of negative sentiment list.
    PARAM sent_scores: dictionary lists as value for keys
    RETURN: edge_list: list of lists
    """
    edge_list = []
    brand_list = sent_scores.keys()
    # print brand_list, len(sent_scores[brand_list[0]])
    for x in range(0, len(brand_list)):
        for y in range(x+1, len(brand_list)):
            positive_sent = []
            negative_sent = []
            for i in range(0, len(sent_scores[brand_list[0]])):
                sent_x = sent_scores[brand_list[x]][i]
                sent_y = sent_scores[brand_list[y]][i]
                # print i, brand_list[x], brand_list[y], sent_x, sent_y
                if (not(np.isnan(sent_x))) & (not(np.isnan(sent_y))):
                    edge_weight = sent_x - sent_y
                    if edge_weight >= 0:
                        positive_sent.append(edge_weight)
                    else:
                        negative_sent.append(edge_weight)
            if len(positive_sent)>0:
                # print positive_sent
                edge_list.append([brand_list[x], brand_list[y], np.mean(positive_sent)])
            if len(negative_sent)>0:
                # print negative_sent
                edge_list.append([brand_list[y], brand_list[x], abs(np.mean(negative_sent))])
    return edge_list

def form_network(edge_list):
    """
    Forms a network
    Calculates unweighted pagerank
    Prints the calculated pagerank values
    PARAM edge_list: list of lists
    RETURN: None
    """
    directed_graph = nx.DiGraph()
    directed_graph.add_weighted_edges_from(edge_list, weight = 'weight')
    # nx.draw(directed_graph)
    # pylab.show()
    # 1-alpha: probability with which the user might click on outgoing link randomly.
    # The mentions in the post of a given brand aren't random, hence alpha is made close to 1
    pagerank_unweighted = nx.pagerank(directed_graph, alpha=0.99, weight = 1)
    print(pagerank_unweighted)
    print(len(pagerank_unweighted))

    
if __name__ == '__main__':
    """
    Reads data and brand list.
    Calculates the sentiment of all brands in each review
    Forms an edge list based on teh sentiment of the comparisons
    Calculates the phage rank of the product
    """
    reviews_data = pd.read_excel('Edmunds Reviews.xlsx', encoding='latin1')
    brand_list = ['ES', 'LS', 'RX', 'A8', 'A6', '3Series', '5Series', '7Series', 'XJ', 'SClass']
    sentiment_scores = sentiment_scores(reviews_data, brand_list)
    edge_list = create_edges(sentiment_scores)
    # print edge_list
    pagerank = form_network(edge_list)
                                  
                                      

IndexError: no such group